In [ ]:
# 1. Load data: passage_embeddings, query_train_embeddings, train_data (a proportion of)
# 2. Network input: 768 output: 32
# 3. Train: concatenate original 768 with 32 

## Import data: load_data.py

In [367]:
import csv
import pickle
import numpy as np

PASSAGE_NP_PATH = "/home/jianx/results/passage_0__emb_p__data_obj_0.pb"
PASSAGE_MAP_PATH = "/datadrive/jianx/data/annoy/100_ance_passage_map.dict"
QUERY_TRAIN_NP_PATH = "/home/jianx/results/query_0__emb_p__data_obj_0.pb"
QUERY_MAP_PATH = "/datadrive/jianx/data/annoy/100_ance_query_train_map.dict"
TRAIN_RANK_PATH = "/datadrive/jianx/data/train_data/ance_training_rank100_200000_random.csv"
# TRAIN_RANK_PATH = "/datadrive/ruohan/reverse_ranker/append/layer_6_optim_fixed/new_train_10000_phase1.csv"


OUT_RANK = 200
N_PASSAGE = 200000
def obj_reader(path):
    with open(path, 'rb') as handle:
        return pickle.load(handle, encoding="bytes")
def load_train(path):
    with open(path, "r") as file:
        pos_dict = {}
        neg_dict = {}
        count = 0
        for line in file:
            if count >= N_PASSAGE * 100:
                break
            count += 1
            tokens = line.split(",")
            pid = int(tokens[0])
            qid = int(tokens[1])
            rank = int(tokens[2].rstrip())
            if rank == 0:
                if pid not in neg_dict:
                    neg_dict[pid] = {}
                neg_dict[pid][qid] = OUT_RANK
            else:
                if pid not in pos_dict:
                    pos_dict[pid] = {}
                pos_dict[pid][qid] = rank
    return pos_dict, neg_dict
def map_id(old_np, mapping):
    new_dict = dict(zip(mapping.values(),old_np))
    return new_dict
def load():
    print("Load embeddings.")
    passage_np = obj_reader(PASSAGE_NP_PATH)
    pid_mapping = obj_reader(PASSAGE_MAP_PATH)
    query_np = obj_reader(QUERY_TRAIN_NP_PATH)
    qid_mapping = obj_reader(QUERY_MAP_PATH)
    print("Mapping ids.")
    query_dict = map_id(query_np, qid_mapping)
    passage_dict = map_id(passage_np, pid_mapping)
    print("Load training data.")
    train_pos_dict, train_neg_dict = load_train(TRAIN_RANK_PATH)
    return train_pos_dict, train_neg_dict, query_dict, passage_dict

In [368]:
train_pos_dict, train_neg_dict = load_train(TRAIN_RANK_PATH)

In [369]:
def count_unique_queries(train_pos, train_neg):
    unique_queries = set()
    for pid, qids in train_pos.items():
        for qid, rank in qids.items():
            unique_queries.add(qid)
    for pid, qids in train_neg.items():
        for qid, rank in qids.items():
            unique_queries.add(qid)
    print(len(unique_queries))
    return unique_queries

In [370]:
count_unique_queries(train_pos_dict, train_neg_dict)

487839


{1048576,
 3,
 4,
 5,
 6,
 8,
 12,
 1048589,
 1048588,
 1048591,
 15,
 1048593,
 18,
 19,
 20,
 16,
 1048595,
 1048599,
 24,
 1048592,
 26,
 1048603,
 1048604,
 1048605,
 1048606,
 1048607,
 31,
 1048609,
 1048611,
 1048612,
 1048613,
 1048614,
 39,
 40,
 1048615,
 42,
 43,
 1048618,
 1048622,
 48,
 51,
 1048627,
 1048629,
 54,
 55,
 1048631,
 59,
 60,
 1048637,
 62,
 63,
 1048640,
 1048641,
 64,
 67,
 68,
 69,
 70,
 1048644,
 1048648,
 14,
 1048650,
 1048651,
 76,
 1048653,
 77,
 79,
 1048655,
 80,
 1048661,
 1048662,
 91,
 1048671,
 1048676,
 100,
 1048677,
 1048679,
 105,
 1048681,
 1048683,
 108,
 1048685,
 1048686,
 1048687,
 112,
 1048689,
 114,
 113,
 111,
 1048693,
 118,
 1048694,
 1048690,
 1048697,
 121,
 1048698,
 125,
 1048703,
 1048704,
 1048710,
 1048711,
 1048713,
 141,
 142,
 144,
 145,
 1048722,
 1048723,
 1048724,
 1048721,
 1048726,
 150,
 152,
 1048729,
 155,
 1048736,
 161,
 160,
 163,
 166,
 1048742,
 168,
 1048743,
 1048745,
 1048748,
 173,
 174,
 1048751,
 176,


In [2]:
train_pos_dict, train_neg_dict, query_dict, passage_dict = load()

Load embeddings.
Mapping ids.
Load training data.


## Network Architecture: network.py

In [307]:
import torch
import torch.nn as nn

NUM_HIDDEN_NODES = 64
NUM_HIDDEN_LAYERS = 3
DROPOUT_RATE = 0.1
FEAT_COUNT = 768


# Define the network
class CorpusNet(torch.nn.Module):

    def __init__(self, embed_size):
        super(CorpusNet, self).__init__()

        layers = []
        last_dim = FEAT_COUNT
        for i in range(NUM_HIDDEN_LAYERS):
            layers.append(nn.Linear(last_dim, NUM_HIDDEN_NODES))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm(NUM_HIDDEN_NODES))
            layers.append(nn.Dropout(p=DROPOUT_RATE))
            last_dim = NUM_HIDDEN_NODES
        layers.append(nn.Linear(last_dim, embed_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    def parameter_count(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


## Train reverse ranker: train.py

In [308]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import torch.nn.functional as F


TOP_K = 100

# With probability alpha
# Select a random negative sample from train_neg_dict
ALPHA = 0.5

def dot_product(A, B, normalize=False):
    if normalize:
        A = F.normalize(A)
        B = F.normalize(B)
    b = A.shape[0]
    embed = A.shape[1]
    result = torch.bmm(A.view(b, 1, embed), B.view(b, embed, 1))
    return result


def mini_batch(batch_size, device, train_pos_dict, train_neg_dict, query_dict, passage_dict):
    passage_list = list(train_neg_dict.keys())
    passages = []
    pos = []
    neg = []
    pos_rank_list = []
    neg_rank_list = []
    while len(passages) < batch_size:
        pid = random.sample(passage_list, 1)[0]
        try:
            temp_pos_list = list(train_pos_dict[pid].keys())
        except:
            continue
        try:
            temp_neg_list = list(train_neg_dict[pid].keys())
        except:
            continue
        if np.random.uniform(0,1,1) <= ALPHA:
            random_positive = random.sample(temp_pos_list, 1)
            pos_qid = random_positive[0]
            pos_rank = train_pos_dict[pid][pos_qid]
            random_negative = random.sample(temp_neg_list, 1)
            neg_qid = random_negative[0]
            neg_rank = train_neg_dict[pid][neg_qid]
#             not_negative = True
#             while not_negative:
#                 temp_neg_qid = random.sample(list(query_dict.keys()), 1)
#                 if temp_neg_qid not in temp_query_list:
#                     neg_qid = temp_neg_qid[0]
#                     neg_rank = 1000
#                     not_negative = False
        else:
            if len(temp_pos_list) < 2:
                continue
            pos_neg_pair = random.sample(temp_pos_list, 2)
            # e.g. 60 >= 3
            if train_pos_dict[pid][pos_neg_pair[0]] >= train_pos_dict[pid][pos_neg_pair[1]]:
                pos_qid = pos_neg_pair[1]
                neg_qid = pos_neg_pair[0]
            # e.g. 3 < 60
            else:
                pos_qid = pos_neg_pair[0]
                neg_qid = pos_neg_pair[1]   
            pos_rank = train_pos_dict[pid][pos_qid]
            neg_rank = train_pos_dict[pid][neg_qid]
        p_seq = passage_dict[pid]
        pos_seq = query_dict[pos_qid]
        neg_seq = query_dict[neg_qid]
        passages.append(p_seq)
        pos.append(pos_seq)
        neg.append(neg_seq)
        pos_rank_list.append(TOP_K - pos_rank)
        neg_rank_list.append(TOP_K - neg_rank)
#         pos_rank_list.append((TOP_K - pos_rank) * 2)
#         neg_rank_list.append((TOP_K - neg_rank) * 2)
    labels = torch.stack([torch.FloatTensor(pos_rank_list), torch.FloatTensor(neg_rank_list)], dim=1)
    passages = torch.from_numpy(np.stack(passages))
    pos = torch.from_numpy(np.stack(pos))
    neg = torch.from_numpy(np.stack(neg))
    return passages.to(device), pos.to(device), neg.to(device), labels.to(device)


def train(net, epoch_size, batch_size, optimizer, device, train_pos_dict, train_neg_dict, 
          query_dict, passage_dict, scale=10, loss_option="ce"):
    bce = nn.BCELoss()
    ce = nn.CrossEntropyLoss()
    softmax = nn.Softmax(dim=1)
    train_loss = 0.0
    net.train()
    for mb_idx in range(epoch_size):
        # Read in a new mini-batch of data!
        passages, pos, neg, labels = mini_batch(batch_size, device, train_pos_dict, train_neg_dict, 
                                                query_dict, passage_dict)
        optimizer.zero_grad()
        p_embed = torch.cat((net(passages), passages), 1).to(device)
        pos_embed = torch.cat((net(pos), pos), 1).to(device)
        neg_embed = torch.cat((net(neg), neg), 1).to(device)
        out_pos = dot_product(p_embed, pos_embed).to(device)
        out_neg = dot_product(p_embed, neg_embed).to(device)
        out = torch.cat((out_pos, out_neg), -1).squeeze()
#         out = torch.cat((out_pos, out_neg), -1) * torch.tensor([scale], dtype=torch.float).to(device)
#         print(softmax(out))
#         print(labels)
#         loss = criterion(softmax(out).squeeze(), softmax(labels))
        if loss_option == "bce":
            loss = bce(softmax(out), softmax(labels))
        if loss_option == "ce":
            loss = ce(out, torch.tensor([0 for i in range(batch_size)]).to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        # print(str(mb_idx) + " iteration: " + str(train_loss / (mb_idx + 1)))
    return train_loss / epoch_size


## Main function: main.py

In [310]:
import datetime
from datetime import datetime, timezone, timedelta

TIME_OFFSET = -4


def print_message(s, offset=TIME_OFFSET):
    print("[{}] {}".format(datetime.now(timezone(timedelta(hours=offset))).strftime("%b %d, %H:%M:%S"), s), flush=True)

In [322]:
import torch
from torch import optim
import csv
import sys
import os


MODEL_PATH = "/datadrive/ruohan/random_sample/"
CURRENT_DEVICE = "cuda:0"
PRETRAINED_PATH = "/datadrive/ruohan/random_sample/reverse_alpha0.5_initial_optim1000_100_1000_0.0001_32.model"

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)


def main(num_epochs, epoch_size, batch_size, learning_rate, model_path, embed_size, pretrained=False):
    if pretrained:
        checkpoint = torch.load(PRETRAINED_PATH)
        net = CorpusNet(embed_size=embed_size)
        net.load_state_dict(checkpoint['model'])
        net.to(CURRENT_DEVICE)
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)
        optimizer.load_state_dict(checkpoint['optimizer'])
    else:
        net = CorpusNet(embed_size=embed_size).to(CURRENT_DEVICE)
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    print("Loading data")
#     train_rank_dict, query_dict, passage_dict = load()
#     print("Data successfully loaded.")
#     print("Positive Negative Pair dict size: " + str(len(train_rank_dict)))
#     print("Num of queries: " + str(len(query_dict)))
#     print("Num of passages: " + str(len(passage_dict)))
#     print("Finish loading.")

    arg_str = "reverse_alpha0.5_new_train" + str(num_epochs) + "_" + str(epoch_size) + "_" + str(batch_size) + "_" + str(learning_rate) + "_" + str(
        embed_size)
    unique_path = model_path + arg_str + ".model"
    output_path = model_path + arg_str + ".csv"
    for ep_idx in range(num_epochs):
        train_loss = train(net, epoch_size, batch_size, optimizer, CURRENT_DEVICE, train_pos_dict, 
                           train_neg_dict, query_dict, passage_dict)
        print_message([ep_idx,train_loss])
        with open(output_path, mode='a+') as output:
            output_writer = csv.writer(output)
            output_writer.writerow([ep_idx, train_loss])
        torch.save({
                "model": net.state_dict(),
                "optimizer": optimizer.state_dict(),
                "n_epoch": ep_idx,
                "train_loss": train_loss,
                "n_hidden_layer": NUM_HIDDEN_LAYERS
                    }, unique_path)


In [323]:
# Train on new training data
main(1000,100,1000,0.0001,MODEL_PATH,32,pretrained=True)

Loading data
[Aug 22, 19:27:30] [0, 0.323385965526104]
[Aug 22, 19:27:34] [1, 0.3231030997633934]
[Aug 22, 19:27:38] [2, 0.32167921364307406]
[Aug 22, 19:27:41] [3, 0.317580509185791]
[Aug 22, 19:27:45] [4, 0.31342725723981857]
[Aug 22, 19:27:48] [5, 0.3092945224046707]
[Aug 22, 19:27:52] [6, 0.3106920599937439]
[Aug 22, 19:27:55] [7, 0.30511011242866515]
[Aug 22, 19:27:59] [8, 0.3064738690853119]
[Aug 22, 19:28:02] [9, 0.30550488233566286]
[Aug 22, 19:28:06] [10, 0.30320884197950365]
[Aug 22, 19:28:09] [11, 0.30246049642562867]
[Aug 22, 19:28:13] [12, 0.3062605628371239]
[Aug 22, 19:28:16] [13, 0.3019219306111336]
[Aug 22, 19:28:20] [14, 0.30037894159555434]
[Aug 22, 19:28:24] [15, 0.30149695426225664]
[Aug 22, 19:28:27] [16, 0.3016085037589073]
[Aug 22, 19:28:31] [17, 0.30186998456716535]
[Aug 22, 19:28:34] [18, 0.3019505187869072]
[Aug 22, 19:28:38] [19, 0.29662841111421584]
[Aug 22, 19:28:41] [20, 0.2943919947743416]
[Aug 22, 19:28:45] [21, 0.29787339821457864]
[Aug 22, 19:28:48] [

[Aug 22, 19:38:25] [183, 0.26192163333296775]
[Aug 22, 19:38:29] [184, 0.2597629038989544]
[Aug 22, 19:38:32] [185, 0.2572663077712059]
[Aug 22, 19:38:36] [186, 0.2603288613259792]
[Aug 22, 19:38:39] [187, 0.26304649367928506]
[Aug 22, 19:38:43] [188, 0.26131474375724795]
[Aug 22, 19:38:46] [189, 0.2591150535643101]
[Aug 22, 19:38:50] [190, 0.26012832909822464]
[Aug 22, 19:38:53] [191, 0.26012004524469373]
[Aug 22, 19:38:57] [192, 0.2596298451721668]
[Aug 22, 19:39:00] [193, 0.25478551611304284]
[Aug 22, 19:39:04] [194, 0.2565312537550926]
[Aug 22, 19:39:07] [195, 0.2586385896801949]
[Aug 22, 19:39:11] [196, 0.259262635409832]
[Aug 22, 19:39:15] [197, 0.2595961655676365]
[Aug 22, 19:39:18] [198, 0.25826975360512733]
[Aug 22, 19:39:22] [199, 0.25744904205203056]
[Aug 22, 19:39:25] [200, 0.25747373327612877]
[Aug 22, 19:39:29] [201, 0.2564536023139954]
[Aug 22, 19:39:32] [202, 0.25595440462231633]
[Aug 22, 19:39:36] [203, 0.2546730867028236]
[Aug 22, 19:39:39] [204, 0.25522208467125895]


[Aug 22, 19:49:11] [364, 0.2477993328869343]
[Aug 22, 19:49:15] [365, 0.2449130070209503]
[Aug 22, 19:49:18] [366, 0.24249736160039903]
[Aug 22, 19:49:22] [367, 0.2454146847128868]
[Aug 22, 19:49:25] [368, 0.24332647636532784]
[Aug 22, 19:49:29] [369, 0.24463726729154586]
[Aug 22, 19:49:32] [370, 0.24562047362327577]
[Aug 22, 19:49:36] [371, 0.2501884062588215]
[Aug 22, 19:49:39] [372, 0.2458841048181057]
[Aug 22, 19:49:43] [373, 0.24845849916338922]
[Aug 22, 19:49:47] [374, 0.2463123981654644]
[Aug 22, 19:49:50] [375, 0.2458084474503994]
[Aug 22, 19:49:54] [376, 0.24500310510396958]
[Aug 22, 19:49:57] [377, 0.24513452515006065]
[Aug 22, 19:50:01] [378, 0.24509488359093667]
[Aug 22, 19:50:04] [379, 0.2442006327211857]
[Aug 22, 19:50:08] [380, 0.24479227393865585]
[Aug 22, 19:50:11] [381, 0.24757680788636208]
[Aug 22, 19:50:15] [382, 0.24330393061041833]
[Aug 22, 19:50:18] [383, 0.245614859610796]
[Aug 22, 19:50:22] [384, 0.24460130870342256]
[Aug 22, 19:50:25] [385, 0.24374644622206687

[Aug 22, 19:59:48] [544, 0.2357652774453163]
[Aug 22, 19:59:52] [545, 0.2408753602206707]
[Aug 22, 19:59:55] [546, 0.235984528362751]
[Aug 22, 19:59:59] [547, 0.23622112825512886]
[Aug 22, 20:00:02] [548, 0.24072929710149765]
[Aug 22, 20:00:06] [549, 0.23876599296927453]
[Aug 22, 20:00:10] [550, 0.24082691803574563]
[Aug 22, 20:00:13] [551, 0.23933559224009515]
[Aug 22, 20:00:17] [552, 0.23880723237991333]
[Aug 22, 20:00:20] [553, 0.2402970926463604]
[Aug 22, 20:00:24] [554, 0.23543404743075372]
[Aug 22, 20:00:27] [555, 0.238014544993639]
[Aug 22, 20:00:31] [556, 0.2383048216998577]
[Aug 22, 20:00:34] [557, 0.24109234064817428]
[Aug 22, 20:00:38] [558, 0.2398465983569622]
[Aug 22, 20:00:41] [559, 0.23717424109578134]
[Aug 22, 20:00:45] [560, 0.2363867013156414]
[Aug 22, 20:00:49] [561, 0.23414125740528108]
[Aug 22, 20:00:52] [562, 0.23757435649633407]
[Aug 22, 20:00:56] [563, 0.23572333976626397]
[Aug 22, 20:00:59] [564, 0.23707400217652322]
[Aug 22, 20:01:03] [565, 0.23735883057117463

[Aug 22, 20:10:39] [724, 0.23410177886486053]
[Aug 22, 20:10:43] [725, 0.23096805855631827]
[Aug 22, 20:10:46] [726, 0.23171427339315415]
[Aug 22, 20:10:50] [727, 0.23208282068371772]
[Aug 22, 20:10:53] [728, 0.2320476196706295]
[Aug 22, 20:10:57] [729, 0.23094765231013298]
[Aug 22, 20:11:00] [730, 0.2319184073805809]
[Aug 22, 20:11:04] [731, 0.23326310083270074]
[Aug 22, 20:11:08] [732, 0.2349804726243019]
[Aug 22, 20:11:11] [733, 0.23350326001644134]
[Aug 22, 20:11:15] [734, 0.22994081765413285]
[Aug 22, 20:11:19] [735, 0.23007041946053505]
[Aug 22, 20:11:23] [736, 0.23295650899410247]
[Aug 22, 20:11:27] [737, 0.2344723568856716]
[Aug 22, 20:11:31] [738, 0.23187000244855882]
[Aug 22, 20:11:34] [739, 0.23419614955782891]
[Aug 22, 20:11:38] [740, 0.23276554957032203]
[Aug 22, 20:11:42] [741, 0.2351913234591484]
[Aug 22, 20:11:46] [742, 0.23487869620323182]
[Aug 22, 20:11:50] [743, 0.23229612722992898]
[Aug 22, 20:11:54] [744, 0.23378044754266739]
[Aug 22, 20:11:58] [745, 0.238661333769

[Aug 22, 20:21:30] [904, 0.22612510845065117]
[Aug 22, 20:21:34] [905, 0.2264735844731331]
[Aug 22, 20:21:37] [906, 0.2311941508948803]
[Aug 22, 20:21:41] [907, 0.2280077350139618]
[Aug 22, 20:21:44] [908, 0.22928619429469108]
[Aug 22, 20:21:48] [909, 0.2317589570581913]
[Aug 22, 20:21:51] [910, 0.2326772330701351]
[Aug 22, 20:21:55] [911, 0.2293343622982502]
[Aug 22, 20:21:59] [912, 0.23128660172224044]
[Aug 22, 20:22:02] [913, 0.22832270249724387]
[Aug 22, 20:22:06] [914, 0.2280155836045742]
[Aug 22, 20:22:09] [915, 0.22927990838885307]
[Aug 22, 20:22:13] [916, 0.22713399916887284]
[Aug 22, 20:22:16] [917, 0.2278798297047615]
[Aug 22, 20:22:20] [918, 0.22802832826972008]
[Aug 22, 20:22:24] [919, 0.22593460023403167]
[Aug 22, 20:22:27] [920, 0.22782189205288886]
[Aug 22, 20:22:31] [921, 0.22410938650369644]
[Aug 22, 20:22:34] [922, 0.22720740750432014]
[Aug 22, 20:22:38] [923, 0.22615124836564063]
[Aug 22, 20:22:41] [924, 0.22947512984275817]
[Aug 22, 20:22:45] [925, 0.229561133235692

In [314]:
# Train on randomly sampled training data that contains 200,000 passages
main(1000,100,1000,0.0001,MODEL_PATH,32)

Loading data
[Aug 21, 21:09:23] [0, 0.5648153483867645]
[Aug 21, 21:09:28] [1, 0.5230460488796234]
[Aug 21, 21:09:32] [2, 0.5147392797470093]
[Aug 21, 21:09:37] [3, 0.5112910515069962]
[Aug 21, 21:09:42] [4, 0.47609696865081785]
[Aug 21, 21:09:47] [5, 0.4552349862456322]
[Aug 21, 21:09:51] [6, 0.44253856301307676]
[Aug 21, 21:09:56] [7, 0.4400656625628471]
[Aug 21, 21:10:01] [8, 0.42331384152173995]
[Aug 21, 21:10:05] [9, 0.422838029563427]
[Aug 21, 21:10:10] [10, 0.4178295969963074]
[Aug 21, 21:10:15] [11, 0.4135330954194069]
[Aug 21, 21:10:19] [12, 0.4091209214925766]
[Aug 21, 21:10:24] [13, 0.4066312849521637]
[Aug 21, 21:10:29] [14, 0.40492887496948243]
[Aug 21, 21:10:34] [15, 0.40419472247362137]
[Aug 21, 21:10:38] [16, 0.4014122235774994]
[Aug 21, 21:10:43] [17, 0.392780020236969]
[Aug 21, 21:10:48] [18, 0.4000295728445053]
[Aug 21, 21:10:53] [19, 0.3950943174958229]
[Aug 21, 21:10:57] [20, 0.3906182834506035]
[Aug 21, 21:11:02] [21, 0.3895644426345825]
[Aug 21, 21:11:07] [22, 0.

[Aug 21, 21:23:46] [183, 0.3355606824159622]
[Aug 21, 21:23:51] [184, 0.33310237497091294]
[Aug 21, 21:23:56] [185, 0.3345948168635368]
[Aug 21, 21:24:01] [186, 0.33738774448633196]
[Aug 21, 21:24:05] [187, 0.33901813328266145]
[Aug 21, 21:24:10] [188, 0.33543183863162995]
[Aug 21, 21:24:15] [189, 0.33190723896026614]
[Aug 21, 21:24:19] [190, 0.3352330520749092]
[Aug 21, 21:24:24] [191, 0.3361675575375557]
[Aug 21, 21:24:29] [192, 0.33788940668106077]
[Aug 21, 21:24:33] [193, 0.33742121517658236]
[Aug 21, 21:24:38] [194, 0.32968717485666277]
[Aug 21, 21:24:43] [195, 0.3342863130569458]
[Aug 21, 21:24:48] [196, 0.3358091202378273]
[Aug 21, 21:24:52] [197, 0.33369277000427244]
[Aug 21, 21:24:57] [198, 0.33352582961320876]
[Aug 21, 21:25:02] [199, 0.33327007681131365]
[Aug 21, 21:25:07] [200, 0.33554099589586256]
[Aug 21, 21:25:11] [201, 0.3315505543351173]
[Aug 21, 21:25:16] [202, 0.3345144659280777]
[Aug 21, 21:25:21] [203, 0.3305322962999344]
[Aug 21, 21:25:25] [204, 0.3359568902850151

[Aug 21, 21:38:01] [364, 0.32384567230939865]
[Aug 21, 21:38:06] [365, 0.3251858603954315]
[Aug 21, 21:38:11] [366, 0.3173109683394432]
[Aug 21, 21:38:15] [367, 0.3234953385591507]
[Aug 21, 21:38:20] [368, 0.3235840791463852]
[Aug 21, 21:38:25] [369, 0.32282782554626466]
[Aug 21, 21:38:29] [370, 0.32247082561254503]
[Aug 21, 21:38:34] [371, 0.3205060064792633]
[Aug 21, 21:38:39] [372, 0.3240305504202843]
[Aug 21, 21:38:44] [373, 0.3219184249639511]
[Aug 21, 21:38:49] [374, 0.32008342415094376]
[Aug 21, 21:38:53] [375, 0.3211660632491112]
[Aug 21, 21:38:58] [376, 0.3221993830800056]
[Aug 21, 21:39:03] [377, 0.3196236178278923]
[Aug 21, 21:39:07] [378, 0.32255692780017853]
[Aug 21, 21:39:12] [379, 0.3181971114873886]
[Aug 21, 21:39:17] [380, 0.3155373504757881]
[Aug 21, 21:39:21] [381, 0.3201099181175232]
[Aug 21, 21:39:26] [382, 0.32272948175668714]
[Aug 21, 21:39:31] [383, 0.3252016460895538]
[Aug 21, 21:39:36] [384, 0.31840686351060865]
[Aug 21, 21:39:40] [385, 0.31760854661464694]
[A

[Aug 21, 21:52:19] [545, 0.3148719310760498]
[Aug 21, 21:52:23] [546, 0.3149619349837303]
[Aug 21, 21:52:28] [547, 0.3169791463017464]
[Aug 21, 21:52:33] [548, 0.3143540874123573]
[Aug 21, 21:52:38] [549, 0.3138352778553963]
[Aug 21, 21:52:42] [550, 0.3166274744272232]
[Aug 21, 21:52:47] [551, 0.3140897038578987]
[Aug 21, 21:52:52] [552, 0.31314496129751207]
[Aug 21, 21:52:57] [553, 0.3157023248076439]
[Aug 21, 21:53:01] [554, 0.3160529610514641]
[Aug 21, 21:53:06] [555, 0.31506718397140504]
[Aug 21, 21:53:11] [556, 0.3185998597741127]
[Aug 21, 21:53:15] [557, 0.31721405118703844]
[Aug 21, 21:53:20] [558, 0.3153876268863678]
[Aug 21, 21:53:25] [559, 0.31502410292625427]
[Aug 21, 21:53:30] [560, 0.31569922387599947]
[Aug 21, 21:53:35] [561, 0.3148640510439873]
[Aug 21, 21:53:39] [562, 0.3110308727622032]
[Aug 21, 21:53:44] [563, 0.31346847891807555]
[Aug 21, 21:53:49] [564, 0.3091960519552231]
[Aug 21, 21:53:53] [565, 0.31399391919374464]
[Aug 21, 21:53:58] [566, 0.3163704711198807]
[Au

[Aug 21, 22:06:34] [726, 0.3106439259648323]
[Aug 21, 22:06:39] [727, 0.3119844228029251]
[Aug 21, 22:06:43] [728, 0.3161603307723999]
[Aug 21, 22:06:48] [729, 0.3099535086750984]
[Aug 21, 22:06:53] [730, 0.31064781367778777]
[Aug 21, 22:06:58] [731, 0.3096667203307152]
[Aug 21, 22:07:02] [732, 0.3131447234749794]
[Aug 21, 22:07:07] [733, 0.3143704631924629]
[Aug 21, 22:07:12] [734, 0.31220449760556224]
[Aug 21, 22:07:17] [735, 0.31104904145002366]
[Aug 21, 22:07:21] [736, 0.31124465584754946]
[Aug 21, 22:07:26] [737, 0.3080854222178459]
[Aug 21, 22:07:31] [738, 0.3091346329450607]
[Aug 21, 22:07:36] [739, 0.3118614336848259]
[Aug 21, 22:07:40] [740, 0.3117482921481132]
[Aug 21, 22:07:45] [741, 0.31196626633405683]
[Aug 21, 22:07:50] [742, 0.3102193269133568]
[Aug 21, 22:07:54] [743, 0.30902865588665007]
[Aug 21, 22:07:59] [744, 0.3140587171912193]
[Aug 21, 22:08:04] [745, 0.3129078185558319]
[Aug 21, 22:08:09] [746, 0.3130745068192482]
[Aug 21, 22:08:13] [747, 0.3134924730658531]
[Aug

[Aug 21, 22:20:51] [907, 0.30939137503504754]
[Aug 21, 22:20:56] [908, 0.31068321853876113]
[Aug 21, 22:21:01] [909, 0.3101643270254135]
[Aug 21, 22:21:05] [910, 0.3048976281285286]
[Aug 21, 22:21:10] [911, 0.30577084600925447]
[Aug 21, 22:21:15] [912, 0.3042797577381134]
[Aug 21, 22:21:20] [913, 0.3100742614269257]
[Aug 21, 22:21:25] [914, 0.3140442457795143]
[Aug 21, 22:21:29] [915, 0.30894653260707855]
[Aug 21, 22:21:34] [916, 0.304944412112236]
[Aug 21, 22:21:39] [917, 0.3080859673023224]
[Aug 21, 22:21:43] [918, 0.3090533998608589]
[Aug 21, 22:21:48] [919, 0.3122329092025757]
[Aug 21, 22:21:53] [920, 0.30997232437133787]
[Aug 21, 22:21:58] [921, 0.3096447369456291]
[Aug 21, 22:22:02] [922, 0.3083242684602737]
[Aug 21, 22:22:07] [923, 0.3059992201626301]
[Aug 21, 22:22:12] [924, 0.3083860436081886]
[Aug 21, 22:22:17] [925, 0.3110584622621536]
[Aug 21, 22:22:22] [926, 0.3106971698999405]
[Aug 21, 22:22:26] [927, 0.307562021613121]
[Aug 21, 22:22:31] [928, 0.30863617569208146]
[Aug 2

In [300]:
# Phase 2 training on new training data based on 10000 passages
main(1000,100,1000,0.003,MODEL_PATH,768)

Loading data
0 0.353420561850071
1 0.3513375648856163
2 0.35419552385807035
3 0.35160008013248445
4 0.36113243877887724
5 0.3553283751010895
6 0.3494171527028084
7 0.34980605870485304
8 0.34332031100988386
9 0.35052149176597597
10 0.34904360234737397
11 0.3417508074641228
12 0.3411302834749222
13 0.3447012734413147
14 0.3452724155783653
15 0.34165508836507796
16 0.34857686191797255
17 0.3331988200545311
18 0.337386055290699
19 0.33225251972675324
20 0.3377408823370934
21 0.33948110193014147
22 0.33716789573431016
23 0.3380432713031769
24 0.331033271253109
25 0.33493414759635926
26 0.32479338616132736
27 0.3359233784675598
28 0.3341635549068451
29 0.33562460154294965
30 0.32903757721185684
31 0.32915620505809784
32 0.33120024353265765
33 0.3341273573040962
34 0.33158291786909105
35 0.328888273537159
36 0.3252687966823578
37 0.32342815816402437
38 0.3297601166367531
39 0.32826134443283084
40 0.32709084898233415
41 0.32339636027812957
42 0.3225845515727997
43 0.3241695249080658
44 0.32725

354 0.2966870865225792
355 0.30139505848288534
356 0.2993417525291443
357 0.30036827832460405
358 0.30419894129037856
359 0.3021270990371704
360 0.2986648526787758
361 0.30026103511452673
362 0.3023384675383568
363 0.3002732829749584
364 0.3026971927285194
365 0.3016728216409683
366 0.3009513318538666
367 0.3030319473147392
368 0.3006239068508148
369 0.29533358722925185
370 0.30112681835889815
371 0.29966717928647996
372 0.3026625427603722
373 0.2987561547756195
374 0.2952802337706089
375 0.3022211095690727
376 0.3012993714213371
377 0.3005387070775032
378 0.30117395997047425
379 0.29879443228244784
380 0.30582305520772934
381 0.29570551857352256
382 0.30225748881697656
383 0.3024990341067314
384 0.29733470499515535
385 0.29954633563756944
386 0.2989535139501095
387 0.3036481001973152
388 0.29583986818790436
389 0.30081854239106176
390 0.30000429674983026
391 0.2964994794130325
392 0.2998927050828934
393 0.2975958317518234
394 0.2987347581982613
395 0.2978732134401798
396 0.29788325428

705 0.2935000219941139
706 0.2939276218414307
707 0.2946247538924217
708 0.2956163391470909
709 0.29410744726657867
710 0.29360476091504095
711 0.29526013880968094
712 0.297654404938221
713 0.2903597432374954
714 0.29361659944057467
715 0.2925334919989109
716 0.29345066487789156
717 0.2908789131045342
718 0.2943095801770687
719 0.291422605663538
720 0.29230157777667043
721 0.29329558938741684
722 0.2921120622754097
723 0.29468672633171084
724 0.2952258011698723
725 0.2945001220703125
726 0.29307651102542875
727 0.2961132489144802
728 0.29239250510931014
729 0.29208623468875883
730 0.29004808828234674
731 0.29192294403910635
732 0.2955639237165451
733 0.2944714638590813
734 0.29558844208717344
735 0.2909469673037529
736 0.29733355656266214
737 0.2930094136297703
738 0.2935651610791683
739 0.2909414187073708
740 0.29752022922039034
741 0.2957981097698212
742 0.29518126815557477
743 0.2901241958141327
744 0.2948103329539299
745 0.2966797038912773
746 0.29656187564134595
747 0.295109062045

In [300]:
# Phase 2 training on new training data based on 10000 passages
main(1000,100,1000,0.003,MODEL_PATH,768)

Loading data
0 0.353420561850071
1 0.3513375648856163
2 0.35419552385807035
3 0.35160008013248445
4 0.36113243877887724
5 0.3553283751010895
6 0.3494171527028084
7 0.34980605870485304
8 0.34332031100988386
9 0.35052149176597597
10 0.34904360234737397
11 0.3417508074641228
12 0.3411302834749222
13 0.3447012734413147
14 0.3452724155783653
15 0.34165508836507796
16 0.34857686191797255
17 0.3331988200545311
18 0.337386055290699
19 0.33225251972675324
20 0.3377408823370934
21 0.33948110193014147
22 0.33716789573431016
23 0.3380432713031769
24 0.331033271253109
25 0.33493414759635926
26 0.32479338616132736
27 0.3359233784675598
28 0.3341635549068451
29 0.33562460154294965
30 0.32903757721185684
31 0.32915620505809784
32 0.33120024353265765
33 0.3341273573040962
34 0.33158291786909105
35 0.328888273537159
36 0.3252687966823578
37 0.32342815816402437
38 0.3297601166367531
39 0.32826134443283084
40 0.32709084898233415
41 0.32339636027812957
42 0.3225845515727997
43 0.3241695249080658
44 0.32725

354 0.2966870865225792
355 0.30139505848288534
356 0.2993417525291443
357 0.30036827832460405
358 0.30419894129037856
359 0.3021270990371704
360 0.2986648526787758
361 0.30026103511452673
362 0.3023384675383568
363 0.3002732829749584
364 0.3026971927285194
365 0.3016728216409683
366 0.3009513318538666
367 0.3030319473147392
368 0.3006239068508148
369 0.29533358722925185
370 0.30112681835889815
371 0.29966717928647996
372 0.3026625427603722
373 0.2987561547756195
374 0.2952802337706089
375 0.3022211095690727
376 0.3012993714213371
377 0.3005387070775032
378 0.30117395997047425
379 0.29879443228244784
380 0.30582305520772934
381 0.29570551857352256
382 0.30225748881697656
383 0.3024990341067314
384 0.29733470499515535
385 0.29954633563756944
386 0.2989535139501095
387 0.3036481001973152
388 0.29583986818790436
389 0.30081854239106176
390 0.30000429674983026
391 0.2964994794130325
392 0.2998927050828934
393 0.2975958317518234
394 0.2987347581982613
395 0.2978732134401798
396 0.29788325428

705 0.2935000219941139
706 0.2939276218414307
707 0.2946247538924217
708 0.2956163391470909
709 0.29410744726657867
710 0.29360476091504095
711 0.29526013880968094
712 0.297654404938221
713 0.2903597432374954
714 0.29361659944057467
715 0.2925334919989109
716 0.29345066487789156
717 0.2908789131045342
718 0.2943095801770687
719 0.291422605663538
720 0.29230157777667043
721 0.29329558938741684
722 0.2921120622754097
723 0.29468672633171084
724 0.2952258011698723
725 0.2945001220703125
726 0.29307651102542875
727 0.2961132489144802
728 0.29239250510931014
729 0.29208623468875883
730 0.29004808828234674
731 0.29192294403910635
732 0.2955639237165451
733 0.2944714638590813
734 0.29558844208717344
735 0.2909469673037529
736 0.29733355656266214
737 0.2930094136297703
738 0.2935651610791683
739 0.2909414187073708
740 0.29752022922039034
741 0.2957981097698212
742 0.29518126815557477
743 0.2901241958141327
744 0.2948103329539299
745 0.2966797038912773
746 0.29656187564134595
747 0.295109062045

In [263]:
# alpha=0.5 embed_size = 32
# Fine tune 0.08 loss model on combined training data
main(1000,100,1000,0.0005,MODEL_PATH,32)

Loading data
0 0.44286092519760134
1 0.41018016517162326
2 0.39681606948375703
3 0.3836226466298103
4 0.3788923037052154
5 0.3795785853266716
6 0.37380550891160963
7 0.3699179375171661
8 0.36546278417110445
9 0.3575321590900421
10 0.3605234757065773
11 0.3559069433808327
12 0.3575423619151115
13 0.3568454205989838
14 0.35904850900173185
15 0.35228013068437575
16 0.3537989324331284
17 0.3565392816066742
18 0.3575398352742195
19 0.35302289843559265
20 0.3531530228257179
21 0.34652263432741165
22 0.34973713010549545
23 0.3485590985417366
24 0.34616458892822266
25 0.34684926867485044
26 0.34428019732236864
27 0.3404961347579956
28 0.3447796058654785
29 0.3378411623835564
30 0.33958410292863844
31 0.33689560830593107
32 0.33597907423973083
33 0.33340404838323595
34 0.3388045984506607
35 0.3361538007855415
36 0.3316317254304886
37 0.3304696390032768
38 0.33800034791231154
39 0.33302733331918716
40 0.33589208871126175
41 0.33240102857351306
42 0.3320202335715294
43 0.33778493136167526
44 0.33

355 0.2857758376002312
356 0.29096112489700315
357 0.2873472094535828
358 0.2866326084733009
359 0.29060936346650124
360 0.2840796275436878
361 0.28587437734007837
362 0.28649509370327
363 0.28447299361228945
364 0.2861778123676777
365 0.289576013982296
366 0.28386409461498263
367 0.2895004163682461
368 0.2845162458717823
369 0.28465771064162254
370 0.28847469240427015
371 0.2836725963652134
372 0.28359140947461126
373 0.2831391094624996
374 0.28632900774478914
375 0.28273219645023345
376 0.2844939889013767
377 0.28694597750902173
378 0.28658697605133054
379 0.28414548844099047
380 0.287706783413887
381 0.28621883764863015
382 0.2840022899210453
383 0.28274840354919434
384 0.2868871572613716
385 0.28776445776224135
386 0.28567948684096334
387 0.2857161957025528
388 0.2908845761418343
389 0.2820793893933296
390 0.2859624692797661
391 0.28224766209721563
392 0.2805400379002094
393 0.2843822480738163
394 0.28681581407785417
395 0.2881316387653351
396 0.28258372366428375
397 0.283663031160

706 0.27591570138931276
707 0.27355753362178803
708 0.27626216903328893
709 0.2742582158744335
710 0.2755006293952465
711 0.2748962651193142
712 0.27511954203248024
713 0.27082902789115904
714 0.2753157976269722
715 0.27762468412518504
716 0.2752727550268173
717 0.2750895734131336
718 0.2755272027850151
719 0.2758322238922119
720 0.27535332053899764
721 0.2723236036300659
722 0.2740972213447094
723 0.2727756610512733
724 0.2786213245987892
725 0.2767795214056969
726 0.27431962221860884
727 0.2766470730304718
728 0.2723151113092899
729 0.27349931225180624
730 0.27629235371947286
731 0.2761450308561325
732 0.28005261197686193
733 0.27720512315630913
734 0.2746708880364895
735 0.27574741378426554
736 0.2788456471264362
737 0.27495072573423385
738 0.2756995816528797
739 0.2730285347998142
740 0.2754156519472599
741 0.2742257945239544
742 0.27775713071227076
743 0.2721721145510674
744 0.275190844386816
745 0.2736282606422901
746 0.27464160934090615
747 0.2778113280236721
748 0.2727000677585

In [263]:
# alpha=0.5 embed_size = 32
# Fine tune 0.08 loss model on combined training data
main(1000,100,1000,0.0005,MODEL_PATH,32)

Loading data
0 0.44286092519760134
1 0.41018016517162326
2 0.39681606948375703
3 0.3836226466298103
4 0.3788923037052154
5 0.3795785853266716
6 0.37380550891160963
7 0.3699179375171661
8 0.36546278417110445
9 0.3575321590900421
10 0.3605234757065773
11 0.3559069433808327
12 0.3575423619151115
13 0.3568454205989838
14 0.35904850900173185
15 0.35228013068437575
16 0.3537989324331284
17 0.3565392816066742
18 0.3575398352742195
19 0.35302289843559265
20 0.3531530228257179
21 0.34652263432741165
22 0.34973713010549545
23 0.3485590985417366
24 0.34616458892822266
25 0.34684926867485044
26 0.34428019732236864
27 0.3404961347579956
28 0.3447796058654785
29 0.3378411623835564
30 0.33958410292863844
31 0.33689560830593107
32 0.33597907423973083
33 0.33340404838323595
34 0.3388045984506607
35 0.3361538007855415
36 0.3316317254304886
37 0.3304696390032768
38 0.33800034791231154
39 0.33302733331918716
40 0.33589208871126175
41 0.33240102857351306
42 0.3320202335715294
43 0.33778493136167526
44 0.33

355 0.2857758376002312
356 0.29096112489700315
357 0.2873472094535828
358 0.2866326084733009
359 0.29060936346650124
360 0.2840796275436878
361 0.28587437734007837
362 0.28649509370327
363 0.28447299361228945
364 0.2861778123676777
365 0.289576013982296
366 0.28386409461498263
367 0.2895004163682461
368 0.2845162458717823
369 0.28465771064162254
370 0.28847469240427015
371 0.2836725963652134
372 0.28359140947461126
373 0.2831391094624996
374 0.28632900774478914
375 0.28273219645023345
376 0.2844939889013767
377 0.28694597750902173
378 0.28658697605133054
379 0.28414548844099047
380 0.287706783413887
381 0.28621883764863015
382 0.2840022899210453
383 0.28274840354919434
384 0.2868871572613716
385 0.28776445776224135
386 0.28567948684096334
387 0.2857161957025528
388 0.2908845761418343
389 0.2820793893933296
390 0.2859624692797661
391 0.28224766209721563
392 0.2805400379002094
393 0.2843822480738163
394 0.28681581407785417
395 0.2881316387653351
396 0.28258372366428375
397 0.283663031160

706 0.27591570138931276
707 0.27355753362178803
708 0.27626216903328893
709 0.2742582158744335
710 0.2755006293952465
711 0.2748962651193142
712 0.27511954203248024
713 0.27082902789115904
714 0.2753157976269722
715 0.27762468412518504
716 0.2752727550268173
717 0.2750895734131336
718 0.2755272027850151
719 0.2758322238922119
720 0.27535332053899764
721 0.2723236036300659
722 0.2740972213447094
723 0.2727756610512733
724 0.2786213245987892
725 0.2767795214056969
726 0.27431962221860884
727 0.2766470730304718
728 0.2723151113092899
729 0.27349931225180624
730 0.27629235371947286
731 0.2761450308561325
732 0.28005261197686193
733 0.27720512315630913
734 0.2746708880364895
735 0.27574741378426554
736 0.2788456471264362
737 0.27495072573423385
738 0.2756995816528797
739 0.2730285347998142
740 0.2754156519472599
741 0.2742257945239544
742 0.27775713071227076
743 0.2721721145510674
744 0.275190844386816
745 0.2736282606422901
746 0.27464160934090615
747 0.2778113280236721
748 0.2727000677585

In [160]:
# Continue to train, to see if loss further decrease
main(1000,100,1000,0.0001,MODEL_PATH,32)

Loading data
0 0.09242286026477814
1 0.09362810656428337
2 0.09357407428324223
3 0.09244837991893291
4 0.0922143154591322
5 0.09312672398984433
6 0.09255419731140137
7 0.09330147758126259
8 0.09183945842087268
9 0.0923061215877533
10 0.09331501424312591
11 0.09389676533639431
12 0.09220728158950806
13 0.0934059551358223
14 0.09242874532938003
15 0.08936845920979977
16 0.09358229465782643
17 0.0932971765100956
18 0.09298205085098743
19 0.08983258441090584
20 0.09362920552492142
21 0.09388238161802293
22 0.09274050042033195
23 0.09257986746728421
24 0.09082861796021462
25 0.09358493655920029
26 0.0940331619232893
27 0.09469860963523388
28 0.09065874092280865
29 0.09436166360974312
30 0.090460539534688
31 0.09320399314165115
32 0.0922974904626608
33 0.09338326439261437
34 0.09289215430617333
35 0.09165102533996106
36 0.09247908197343349
37 0.09109776228666305
38 0.09297090299427509
39 0.09061546042561532
40 0.09364765845239162
41 0.09124253802001477
42 0.0920777302980423
43 0.091157613843

348 0.08840005818754434
349 0.08758257932960987
350 0.08764669790863991
351 0.08852054990828037
352 0.087467403113842
353 0.08855969846248626
354 0.0873168183863163
355 0.08884152792394161
356 0.08982530929148197
357 0.08833487659692764
358 0.08696822308003903
359 0.08612955443561077
360 0.08623002678155899
361 0.08688206970691681
362 0.08616493996232748
363 0.08994153238832951
364 0.08651953391730785
365 0.08965283393859863
366 0.08663481146097184
367 0.08650971673429013
368 0.08885869033634662
369 0.08760820113122464
370 0.08772467516362667
371 0.0860393825173378
372 0.08702977031469344
373 0.08879687972366809
374 0.08629834465682507
375 0.08494268119335174
376 0.08778778035193682
377 0.08750556483864784
378 0.08823110021650792
379 0.08798402734100819
380 0.08940647959709168
381 0.09006117597222328
382 0.0882234325632453
383 0.08694288432598114
384 0.08867004103958606
385 0.08724404513835907
386 0.0855836496502161
387 0.08738089181482792
388 0.08809353031218052
389 0.0878038528561592

692 0.08593089610338212
693 0.08439540445804596
694 0.08479158908128738
695 0.08597571238875389
696 0.08609658271074296
697 0.08410146668553352
698 0.08507859349250793
699 0.08612381160259247
700 0.08458093494176865
701 0.08359806101769209
702 0.08380773246288299
703 0.08342213846743107
704 0.0840354061126709
705 0.0852862548828125
706 0.08400484375655651
707 0.0832472226768732
708 0.08289869219064712
709 0.08572328556329012
710 0.08340738736093044
711 0.0842294618114829
712 0.08411695286631585
713 0.0826189986616373
714 0.0855903384834528
715 0.08457422606647015
716 0.08501401081681252
717 0.08355985596776008
718 0.08488374285399913
719 0.08419502928853034
720 0.08384193021804094
721 0.0843980248272419
722 0.08293031550943851
723 0.086374884955585
724 0.08400150552392006
725 0.0854254986345768
726 0.08551552921533584
727 0.08387076668441296
728 0.0857486842572689
729 0.08460838854312897
730 0.08115911364555359
731 0.0844218560308218
732 0.08391157917678356
733 0.08418139293789864
734 

In [154]:
main(1000,100,1000,0.0001,MODEL_PATH,32)

Loading data
0 0.56778594404459
1 0.5361327835917473
2 0.513755610883236
3 0.4606933206319809
4 0.4318121728301048
5 0.39857877910137174
6 0.38456533759832384
7 0.3615595731139183
8 0.35005480200052264
9 0.3339472904801369
10 0.3274097794294357
11 0.31392027348279955
12 0.3061116561293602
13 0.2963147297501564
14 0.28868519350886346
15 0.28368862375617027
16 0.2772076211869717
17 0.27109444692730905
18 0.2638201983273029
19 0.2643818971514702
20 0.2585298290848732
21 0.25518435552716257
22 0.24759986594319344
23 0.24508200466632843
24 0.2442743219435215
25 0.23841279581189156
26 0.23553462833166122
27 0.23538652062416077
28 0.23230824798345565
29 0.22786283761262893
30 0.2267448529601097
31 0.2245577844977379
32 0.222188633531332
33 0.21788629859685898
34 0.21683309748768806
35 0.21622372403740883
36 0.21220046132802964
37 0.2103882522881031
38 0.21280795738101005
39 0.2088419534265995
40 0.20935980409383773
41 0.2038043324649334
42 0.20264502257108688
43 0.2026878772675991
44 0.200514

350 0.11572491951286792
351 0.1157834180444479
352 0.11541165180504322
353 0.11835237592458725
354 0.11572006590664387
355 0.11635373078286648
356 0.11516583666205406
357 0.11767843320965767
358 0.1159132120013237
359 0.11479521587491036
360 0.11769385948777199
361 0.11547223642468453
362 0.11416971199214458
363 0.11566571094095707
364 0.11675478801131249
365 0.11468841753900051
366 0.11626176193356513
367 0.11194879800081253
368 0.11638198189437389
369 0.11616038069128991
370 0.11652151018381118
371 0.11586239628493786
372 0.11611321739852429
373 0.11362403310835362
374 0.11323442600667477
375 0.11348753213882447
376 0.11244064010679722
377 0.11321325078606606
378 0.11660580858588218
379 0.11450220353901386
380 0.11158235982060433
381 0.11370470486581326
382 0.112759865000844
383 0.11527853228151798
384 0.11476400703191757
385 0.11305238097906113
386 0.11304062373936176
387 0.11538522511720657
388 0.11602147392928601
389 0.11523488625884055
390 0.11493959866464137
391 0.11406769484281

694 0.10067254401743413
695 0.09762756079435349
696 0.10070199310779572
697 0.09989629112184048
698 0.1007743313908577
699 0.1019477216899395
700 0.09909678183495998
701 0.09973478391766548
702 0.09986857138574123
703 0.0987181182205677
704 0.10089762762188911
705 0.10176450788974761
706 0.09881762765347958
707 0.09976345762610435
708 0.0985382516682148
709 0.0997630337625742
710 0.09871149957180023
711 0.1007556925714016
712 0.09974215231835842
713 0.10064550548791885
714 0.09863779850304127
715 0.09799405805766583
716 0.0999866197258234
717 0.09994803309440613
718 0.10101903811097145
719 0.09954906791448594
720 0.10006252840161324
721 0.10046975664794446
722 0.09902587212622166
723 0.09939764641225338
724 0.09887002363801002
725 0.09785522900521755
726 0.09995790056884289
727 0.09976165272295474
728 0.09825257569551468
729 0.09849015414714814
730 0.09816409789025783
731 0.10063843853771687
732 0.10002238862216473
733 0.09941036738455296
734 0.09705572806298733
735 0.0995917834341526
